## 초기 system prompt 설정 및 벡터 store 생성

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

permission = "backend"

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embedding_function=embedding_model)

## 초기 질문 리스트 가져오기

In [3]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal(f"./company_questions_{permission}.jsonl")
initial_questions

['백엔드팀 주간 업무 계획서에서 서비스 안정성 향상에 대한 구체적인 목표는 무엇인가요?',
 '백엔드팀 주간 업무 계획서의 기능 개발 파트에서 신규 기능 개발을 위한 요구사항 정리는 어떻게 진행되나요?',
 '백엔드팀 주간 업무 계획서의 버그 수정 파트에서 기존 버그 리스트 검토는 어떤 기준으로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 성능 최적화 파트에서 성능 저하 원인 분석은 어떤 방법으로 진행되나요?',
 '백엔드팀 주간 업무 계획서의 팀 회의 및 소통 파트에서 주간 팀 미팅 일정 조율은 어떻게 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 체크리스트에서 성능 최적화 작업 진행 여부 확인은 어떤 기준으로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 검증 포인트에서 신규 기능이 요구사항에 맞게 동작하는지 확인하는 방법은 무엇인가요?',
 '백엔드팀 주간 업무 계획서의 다음 개정 제안에서 업무 계획서의 주간 목표 및 주요 업무 항목을 어떻게 구분할 계획인가요?',
 '백엔드팀 주간 업무 계획서의 기능 구현 및 코드 리뷰 진행은 어떤 절차로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 단위 테스트 작성 및 통과 확인은 어떤 기준으로 진행되나요?',
 '백엔드팀 주간 업무 계획서의 수정된 버그에 대한 테스트 및 검증은 어떤 방식으로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 성능 모니터링 도구 설정 및 데이터 수집은 어떤 도구를 사용하나요?',
 '백엔드팀 주간 업무 계획서의 최적화 작업 진행 후 결과 확인은 어떤 지표를 통해 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 각 팀원 진행 상황 공유는 어떤 형식으로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 다음 주 업무 계획 논의는 어떤 방식으로 진행되나요?',
 '백엔드팀 주간 업무 계획서의 버그 수정 결과 문서화는 어떤 형식으로 이루어지나요?',
 '백엔드팀 주간 업무 계획서의 성능 최적화 후 시스템 응답 시간 및 자원 사용량 확인은 어떻게 진행되나요?',
 '백엔드

In [4]:
def question_chain_setting():
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
            다음 제공되는 대화 내역을 기반으로 다음에 나올 수 있는 사용자가 궁금해할 만한 후속 질문 하나를 사용자 입장에서 질문하세요.

            대화 내역: {history}

            절대로 AI입장에서의 질문을 만들지 마세요.
            ex) 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.
            ex) 추가적으로 궁금한 사항이 있으신가요?
            ex) 데이터 접근 권한 관리 방침에 대해 더 궁금한 사항이 있으면 말씀해 주세요.
            ex) 안녕하세요. 다른 궁금한 점이 있으신가요?
            ex) 안녕하십니까? 도움이 필요하신 부분이 있으신가요?
            ex) ~~ 예시는 다음과 같습니다.


            답변형태는 다음과 같이 해주세요.
            ex) 빌드 결과물의 확인 방법이나 배포 과정에 대해 알고 싶어.
            ex) 문서화 시 유의사항 알려줘.
            ex) 빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 뭐야?
            ex) 모델 개발 & 성능 보고서에서 사용된 프로그래밍 언어는 무엇이야?

            <추가 지시사항>
            이전에 나왔던 질문은 최대한 만들지 말고, 연결된 질문으로 조금이라도 다른 질문을 만들어주세요.

        """
    )

    question_chain = basic_prompt | llm | StrOutputParser()

    return question_chain

In [5]:
question_chain = question_chain_setting()

In [6]:
# tool_prompt = f"사용자는 {permission}팀이며 반드시 {permission}_search 툴을 호출하세요."
tool_prompt = f"사용자는 {permission}(백엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
8. {tool_prompt}
"""

SYSTEM_PROMPT = {
    "role": "system",
    "content": system_message
}

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q
    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})
        print(current_q)

        tool_call = f"<tool_call>{{\"name\": \"{permission}_search\", \"arguments\": {{\"keyword\": \"{current_q}\"}}}}</tool_call>"

        messages.append({"role": "assistant", "content": tool_call})

        # (2) 문서 검색 (vectorstore에서)
        docs = vectorstore.similarity_search(current_q, k=4)
        ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
        ref_text = f"검색 결과:\n-----\n{ref_text}"
        tool_res = f"<tool_response>{ref_text}</tool_response>"
        messages.append({"role": "user", "content": tool_res})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0
        )
        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):

            followup = question_chain.invoke({
                "history": messages
            })
            current_q = followup
        else:
            break

    return {"messages": messages}


In [ ]:
import time
from tqdm import tqdm

train_dataset = []
for q in tqdm(initial_questions):
    time.sleep(1)  # 1초 대기
    sample = generate_one_chat_sample(q, max_turns=4)
    train_dataset.append(sample)

# 저장
with open(f"qwen3_company_train_dataset_{permission}.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)


  0%|          | 0/325 [00:00<?, ?it/s]

백엔드팀 주간 업무 계획서에서 서비스 안정성 향상에 대한 구체적인 목표는 무엇인가요?
장애 대응 프로세스 개선안에는 어떤 구체적인 변경 사항이 포함되어 있는지 알려줘.
장애 대응 훈련이나 시뮬레이션은 구체적으로 어떻게 진행되는지 알려줘.


  0%|          | 1/325 [00:19<1:43:40, 19.20s/it]

백엔드팀 주간 업무 계획서의 기능 개발 파트에서 신규 기능 개발을 위한 요구사항 정리는 어떻게 진행되나요?


  1%|          | 2/325 [00:23<57:09, 10.62s/it]  

...

문제 해결 단계의 문서화에서 문서화된 내용을 팀 내에서 공유하는 이유는 무엇인가요?
문서화된 내용을 팀 내에서 공유할 때 사용하는 구체적인 방법이나 도구가 있다면 알려줘.


 65%|██████▌   | 212/325 [31:04<17:45,  9.43s/it]

배포 단계의 수동 배포 시 배포 스크립트를 사용하는 이유는 무엇인가요?


 66%|██████▌   | 213/325 [32:25<58:01, 31.08s/it]

운영 단계의 이상 징후 발견 시 즉시 대응하는 이유는 무엇인가요?
이상 징후를 발견했을 때 구체적으로 어떤 절차로 대응해야 하는지 알려줘.
